In [0]:
# Lenguaje del Notebook: Python
# VERSIÓN 14: Unificador de Evidencias (Idempotente y Transaccional)

# MAGIC %md
# MAGIC # 📥 5. Unificar Evidencias (Staging -> Final) (v14)
# MAGIC 
# MAGIC Este notebook se ejecuta DESPUÉS de todas las tareas paralelas de validación.
# MAGIC Es idempotente y seguro ante fallos.
# MAGIC 
# MAGIC **Lógica (Transaccional):**
# MAGIC 1.  `main()` orquesta el proceso.
# MAGIC 2.  Lee `table_config` para encontrar todas las tablas `staging_evidences_table`.
# MAGIC 3.  Las une dinámicamente (`unionByName`) para crear un DataFrame `df_staging_completo`.
# MAGIC 4.  **Anti-Duplicación:** Obtiene los `execution_id`s de staging y los cruza con
# MAGIC     `dq_evidences` para encontrar los IDs que *aún no* han sido procesados.
# MAGIC 5.  Filtra `df_staging_completo` para quedarse solo con los datos nuevos.
# MAGIC 6.  **`APPEND`** los datos nuevos a la tabla final `dq_evidences`.
# MAGIC 7.  **Limpieza:** Si el `APPEND` es exitoso, `DELETE` los `execution_id`s procesados
# MAGIC     de todas las tablas de staging.

# COMMAND ----------

# DBTITLE 1, 1. Imports y Constantes
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, broadcast
from functools import reduce
from delta.tables import DeltaTable

# --- Widgets ---
#dbutils.widgets.text("table_id", "", "Id de la tabla a validar")
dbutils.widgets.text("table_name", "maestro_demo", "Id de la tabla a validar")
dbutils.widgets.text("catalog_name", "workspace", "Catálogo de UC donde residen las tablas")
dbutils.widgets.text("schema_name", "dq_framework", "Esquema de UC donde residen las tablas")
# COMMAND ----------

# DBTITLE 2, 2. Carga de librerías y definición de constantes/mapas

CATALOG = dbutils.widgets.get("catalog_name")
SCHEMA = dbutils.widgets.get("schema_name")

TABLE_CONFIG = f"{CATALOG}.{SCHEMA}.dq_tables_config"
EVIDENCES_TABLE = f"{CATALOG}.{SCHEMA}.dq_evidences"


# COMMAND ----------

# DBTITLE 2, 2. Función Principal (main)
def main():
    """
    Flujo principal de unificación de evidencias (v14).
    """
    print("--- Iniciando Script de Unificación de Evidencias (v14) ---")
    
    try:
        # --- 1. Leer la Configuración Maestra ---
        print(f"Leyendo la configuración de maestros desde: {TABLE_CONFIG}")
        
        master_configs = (spark.table(TABLE_CONFIG)
                          .select("table_name", "staging_evidences_table")
                          .collect())
        
        if not master_configs:
            raise Exception(f"No se encontró configuración de maestros en {TABLE_CONFIG}")

        # --- 2. Recolectar todos los DataFrames de Staging ---
        dfs_to_union = []
        staging_table_paths = [] # Guardar las rutas para la limpieza final
        
        print(f"Recolectando datos de {len(master_configs)} tablas de staging...")

        
        if spark.catalog.tableExists(full_table_name):
            print(f"✅ La tabla {full_table_name} existe")
        else:
            print(f"⚠️ La tabla {full_table_name} NO existe")

        for config in master_configs:
            staging_table = config.staging_evidences_table
            staging_table_path = f"{catalog}.{schema}.{staging_table}"
            staging_table_paths.append(staging_table_path) # Añadir a la lista para la limpieza
            
            if spark.catalog.tableExists(staging_table_path):
                print(f"  > Leyendo datos de: {staging_table_path}")
                df_staging = spark.table(staging_table_path)
                
                if not df_staging.isEmpty():
                    dfs_to_union.append(df_staging)
                else:
                    print(f"  > AVISO: La tabla {staging_table_path} existe pero está vacía. Saltando.")
            else:
                print(f"  > AVISO: No se encontró la tabla {staging_table_path}. Saltando.")

        # --- 3. Unificar los DataFrames ---
        if not dfs_to_union:
            print("No se encontraron datos en ninguna tabla de staging. No hay nada que unificar.")
            return 0 # Finaliza con éxito, 0 registros procesados

        print(f"Unificando {len(dfs_to_union)} DataFrames de staging usando 'unionByName'...")
        
        #versión previa, compleja
        '''
        df_staging_completo = reduce(
            lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), 
            dfs_to_union
        ).distinct() # Deduplicar por si acaso el orquestador se re-ejecutó
        '''

        # Inicializamos el DataFrame final como vacío
        df_staging_completo: DataFrame = None

        for df in dfs_to_union:
            if df_staging_completo is None:
                # La primera iteración, asignamos el primer DataFrame
                df_staging_completo = df
            else:
                # Union por nombre de columnas, rellenando columnas faltantes
                df_staging_completo = df_staging_completo.unionByName(df, allowMissingColumns=True)

        # Eliminamos duplicados
        df_staging_completo = df_staging_completo.distinct()#.cache()

        # --- 4. Lógica Anti-Duplicación (Idempotencia) ---
        print("Buscando ejecuciones ya procesadas para evitar duplicados...")
        
        # Obtenemos todos los execution_id que han llegado a staging
        exec_ids_in_staging = (df_staging_completo
                               .select("execution_id")
                               .distinct()
                              )
        
        # Obtenemos los execution_id que YA están en la tabla final
        df_processed_ids = (spark.table(EVIDENCES_TABLE)
                            .select("execution_id")
                            .distinct()
                           )
        
        # Filtramos 'exec_ids_in_staging' para quedarnos solo con los que
        # NO están en 'df_processed_ids' (LEFT_ANTI JOIN)
        df_unprocessed_ids = exec_ids_in_staging.join(
            broadcast(df_processed_ids), # Broadcast para optimizar el join
            "execution_id",
            "left_anti"
        )#.cache()
        
        unprocessed_count = df_unprocessed_ids.count()
        if unprocessed_count == 0:
            print("No hay ejecuciones nuevas en staging. Los datos ya fueron procesados en un run anterior.")
            # (Procedemos a la limpieza por si acaso)
        else:
            print(f"Se han encontrado {unprocessed_count} ejecuciones nuevas para procesar.")
        
        # Filtramos el DataFrame de staging completo para quedarnos solo con los datos nuevos
        df_datos_nuevos = df_staging_completo.join(
            broadcast(df_unprocessed_ids),
            "execution_id",
            "inner"
        )
        
        total_rows = df_datos_nuevos.count()
        
        # --- 5. Escribir en la Tabla Final de Evidencias ---
        if total_rows > 0:
            print(f"Añadiendo (APPEND) {total_rows} registros nuevos a la tabla final: {EVIDENCES_TABLE}...")
            
            (df_datos_nuevos.write
             .format("delta")
             .mode("append")
             .option("mergeSchema", "true")
             .saveAsTable(EVIDENCES_TABLE))
            
            print(f"¡Unificación completada! Se añadieron {total_rows} filas a {EVIDENCES_TABLE}.")
        else:
            print("No se encontraron registros nuevos para añadir a la tabla final.")

        # --- 6. Limpieza Segura de Staging ---
        # Borramos SOLO los IDs que hemos procesado (o que ya estaban procesados)
        # de las tablas de staging.
        exec_ids_to_delete = exec_ids_in_staging.select("execution_id").collect()
        
        if exec_ids_to_delete:
            delete_condition = "execution_id IN ({})".format(
                ",".join([f"'{row.execution_id}'" for row in exec_ids_to_delete])
            )
            
            print(f"Limpiando {len(exec_ids_to_delete)} ejecuciones de las tablas de staging...")
            
            for table_path in staging_table_paths:
                if spark.catalog.tableExists(table_path):
                    try:
                        delta_staging_table = DeltaTable.forName(spark, table_path)
                        delta_staging_table.delete(condition = delete_condition)
                        print(f"  > Limpieza exitosa de: {table_path}")
                    except Exception as e:
                        print(f"  > AVISO: Fallo al limpiar la tabla de staging {table_path}. Error: {e}")
                        # No lanzamos error, el próximo run lo volverá a intentar
            
            print("Limpieza de tablas de staging completada.")

        #df_unprocessed_ids.unpersist()
        return total_rows

    except Exception as e:
        print(f"Error fatal durante la unificación de evidencias: {e}")
        if 'df_unprocessed_ids' in locals() and df_unprocessed_ids.is_cached:
            df_unprocessed_ids.unpersist()
        raise e

# COMMAND ----------

# DBTITLE 3, 3. Punto de Entrada de Ejecución
if __name__ == "__main__":
    try:
        rows_processed = main()
        dbutils.notebook.exit(f"Éxito: {rows_processed} evidencias nuevas unificadas y añadidas.")
    except Exception as e:
        dbutils.notebook.exit(f"Fallo en la unificación de evidencias: {e}")